In [1]:
import pandas as pd

In [2]:
books = pd.read_csv("./raw/shamela_books/Finished.csv").drop(columns=["Unnamed: 0", 'book'])

In [3]:
aranji = pd.read_excel("./processed/english_shamela.xlsx").drop(columns=["Unnamed: 0"])

In [4]:
books['unclassical'] = aranji.loc[:, 'عربي']

In [5]:
books

,en,ar,unclassical
0,[Chapter on entering upon kings] [Entering of...,[باب في الدخول على الملوك] [دخول الأشراف] إن ك...,[باب الدخول على الملوك] [دخول الأشراف] إذا كا...
1,"If he approached him, he would come close to ...",فإن استدناه، قرب منه، فأكب على أطرافه يقبلها، ...,فإذا اقترب منه دنا منه فاتكأ على أعضائه فقبله...
2,"If he signals to him to sit, he sits.",فإن أومأ إليه بالقعود، قعد.,إذا أشار إليه بالجلوس، يجلس.
3,"If he spoke to him, he would answer him in a ...",فإن كلمه، أجابه بانخفاض صوتٍ، وقلة حركة.,إذا تكلم معه أجابه بصوت منخفض وحركة قليلة.
4,"If he is silent, he should get up immediately...",وإن سكت، نهض من ساعته، قبل أن يتمكن به مجلسه ب...,إذا كان صامتاً، فعليه أن ينهض فوراً، قبل أن ي...
...,...,...,...
13565,"In reality, this is only by the will of the O...",وأما في الحقيقة فذلك إنما هو بإرادة من لا يعتر...,والحقيقة أن هذا لا يكون إلا بمشيئة من لا ينسى...
13566,"All praise and thanks be to God, we have compl...",كمل ذلك ولله الحمد والمنّة ونسأله السّلامة في ...,ولله الحمد والمنة، تممنا ذلك، ونسأله السلامة ...
13567,"Version M: At the end, all praise and thanks ...",النسخة م: آخره ولله الحمد والمنّة وصلى الله عل...,النسخة م: وأخيراً الحمد لله والشكر، وصلى الله ...
13568,"It was completed in the month of Dhul-Qi'dah,...",تمت في تاريخ شهر ذي القعدة الحرام بعد ما خلت ث...,تم ذلك في شهر ذي القعدة، بعد ثمانية أيام من ه...


In [13]:
from datasets import Dataset
books = books.rename(columns={"en": "prompt", "ar": "chosen", "unclassical": "rejected"})
ds = Dataset.from_pandas(books)

In [18]:
import evaluate
metric = evaluate.load("AbdulmohsenA/classicier")

def check(row):
    score = metric.compute(texts=[row['chosen'], row['rejected']])['classical_score']
    return score[0] > 0.5 and score[1] < 0.01

In [20]:
new_df = books[books.apply(check, axis=1)]

In [35]:
new_df = new_df[new_df['prompt'].str.len() < 200]
new_df = new_df[new_df['chosen'].str.len() < 200]
new_df = new_df[new_df['rejected'].str.len() < 200]

In [36]:
new_df.reset_index(inplace=True, drop=True)

In [38]:
dpo = Dataset.from_pandas(new_df, preserve_index=False)
dpo.push_to_hub("Abdulmohsena/Pairs_DPO")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/348 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Abdulmohsena/Pairs_DPO/commit/d44b4404096d55b33c8a8354a4871e5e81cd472e', commit_message='Upload dataset', commit_description='', oid='d44b4404096d55b33c8a8354a4871e5e81cd472e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Abdulmohsena/Pairs_DPO', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Abdulmohsena/Pairs_DPO'), pr_revision=None, pr_num=None)

In [39]:
texts = {'prompt': [], 'completion': [], 'label':[]}

for _, row in new_df.iterrows():
    eng = row['prompt']
    true_ar = row['chosen']
    bad_ar = row['rejected']
    
    texts['prompt'].append(eng)
    texts['completion'].append(true_ar)
    texts['label'].append(True)
    
    texts['prompt'].append(eng)
    texts['completion'].append(bad_ar)
    texts['label'].append(False)

In [40]:
kto = Dataset.from_pandas(pd.DataFrame(texts), preserve_index=False)

In [41]:
kto.push_to_hub("Abdulmohsena/Pairs_KTO")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/348 [00:00<?, ?B/s]

d:\anaconda\envs\Faseeh\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\datasets--Abdulmohsena--Pairs_KTO. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/Abdulmohsena/Pairs_KTO/commit/91b95b400257fb9c948942d377130d651fb083de', commit_message='Upload dataset', commit_description='', oid='91b95b400257fb9c948942d377130d651fb083de', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Abdulmohsena/Pairs_KTO', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Abdulmohsena/Pairs_KTO'), pr_revision=None, pr_num=None)

In [28]:
new_df

,en,ar,label
2,"If he approached him, he would come close to ...",فإن استدناه، قرب منه، فأكب على أطرافه يقبلها، ...,True
3,"If he approached him, he would come close to ...",فإذا اقترب منه دنا منه فاتكأ على أعضائه فقبله...,False
4,"If he signals to him to sit, he sits.",فإن أومأ إليه بالقعود، قعد.,True
5,"If he signals to him to sit, he sits.",إذا أشار إليه بالجلوس، يجلس.,False
8,"If he is silent, he should get up immediately...",وإن سكت، نهض من ساعته، قبل أن يتمكن به مجلسه ب...,True
...,...,...,...
27134,"Version M: At the end, all praise and thanks ...",النسخة م: آخره ولله الحمد والمنّة وصلى الله عل...,True
27136,"It was completed in the month of Dhul-Qi'dah,...",تمت في تاريخ شهر ذي القعدة الحرام بعد ما خلت ث...,True
27137,"It was completed in the month of Dhul-Qi'dah,...",تم ذلك في شهر ذي القعدة، بعد ثمانية أيام من ه...,False
27138,"The doctor, professor, Salman Qattaya, the Syr...",يقول الدكتور الطبيب الأستاذ: سلمان قطاية السور...,True
